<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/Matem%C3%A1ticas_CD/Calculo_Opt_Problema_del_Viajante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Utilizaremos programación lineal para resolver el **Problema del Viajante** (Travel Salesman Problem).

El problema es encontrar la **ruta más corta que conecta a todas las ciudades**, con las restricciones de que debe ser una única ruta (i.e. no puede haber sub rutas) y sólo se puede entrar y salir de cada ciudad una sola vez.

Se crearán las "coordenadas" de ciudades ficticias y para medir su distancia se usará la distancia euclideana. 

# Dependencias

In [ ]:
!pip install mip

In [ ]:
### That will kill the current Python runtime process
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install -U plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from scipy.spatial import distance_matrix

from itertools import product
from mip import Model, xsum, minimize, BINARY

import plotly.express as px
import plotly.io as pio

In [ ]:
pio.templates.default = "plotly_white"

# Funciones para la visualización de los datos

In [ ]:
def plot_cities(coords):
    fig = px.scatter(coords.reset_index(), 'x', 'y', hover_name='index')
    fig.update_traces(marker=dict(size=15,
                                line=dict(width=2,
                                            color='DarkSlateGrey')),
                    selector=dict(mode='markers')
                    )
    return fig

def plot_cities_and_route(points, edges):
    fig = plot_cities(points)
    for edge in edges:
        fig.add_shape(
            type = 'line', 
            x0 = points.loc[f'ciudad_{edge[0]}', 'x'], 
            x1 = points.loc[f'ciudad_{edge[1]}', 'x'], 
            y0 = points.loc[f'ciudad_{edge[0]}', 'y'], 
            y1 = points.loc[f'ciudad_{edge[1]}', 'y'], 
            line = dict(color = 'rgb(0, 0, 0)'), 
            opacity = .09,
            )
        fig.add_scatter(x = [mid_points[edge[0]][edge[1]][0]], y = [mid_points[edge[0]][edge[1]][1]], text = [d[edge[0]][edge[1]]], mode='text')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='top right')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', showlegend=False)
    fig.show()

In [ ]:
def get_middle_points(coords):
    return [[((x[0]+y[0])/2, (x[1]+y[1])/2) for _, y in coords.iterrows()] for _, x in coords.iterrows()]

# Datos

Primero crearemos las ciudades y sus coordenadas

In [ ]:
np.random.seed(10)

n = 10 # número de ciudades

points = pd.DataFrame(np.random.randint(0, 30, (n, 2)), columns = ['x', 'y'], index = [f'ciudad_{i}' for i in range(n)])  # coordenadas en el plano cartesiano de cada ciudad
points

Obtenemos el punto medio entre cada ciudad

In [ ]:
mid_points = get_middle_points(points)

In [ ]:
mid_points

Creamos la matriz de distancias

In [ ]:
d = distance_matrix(points, points)  # Distancia entre las ciudades
d

## Visualización de los datos

In [ ]:
plot_cities_and_route(points, product(range(n), range(n))) # Visualización de las ciudades

6 -> 5 -> 2 -> 1 -> ....

## TSP (Travelling salesman problem) con programación lineal

Considere $n$ puntos, $V = \left\{0, 1, \dots, n-1 \right\}$, y la matriz de distancia $D_{n\times n}$ con entradas $d_{i, j} \in \mathbb{R^+}$.

La variable $x_{i,j}$ es tal que 
$$  x_{i,j} = \begin{cases} 1 & \text{ el camino va de la ciudad } i \text{ a la ciudad } j\text{,} \\ 0 & \text{caso contrario.} \end{cases} $$

Tomando una variable dummy $y_i$ que guarda información del orden en que se visitan las ciudades, a partir de la ciudad $0$. Esto se escribe como $ y_i < y_j$ si la ciudad $i$ se visita antes que la ciudad $j$.

Programación lineal favorece la desigualdades no-estrictas respecto a la estrictas, por lo que se puede imponer una condición parecida a la siguiente

$$ y_j \geqslant y_i + x_{i,j} $$ 

cuando $x_{i,j} = 1$ (NOTESE que no es lo mismo que tomar solo $ y_j \geqslant y_i + x_{i,j}$ en el caso general de $x_{i,j}$)


La solución es un conjunto de $n$ pares de puntos indicando la ciudad de salida y ciudad de llegada. Considerando las restricciones que se mencionaron al inicio, tenemos que

Minimizar:
$$\sum_{i\in V, j\in V} d_{i, j}x_{i, j}$$
Sujeto a:
\begin{align}
\sum_{i\in V\setminus \{j\}} x_{i, j} = 1 && \forall j \in V \\
\sum_{j\in V\setminus \{i\}} x_{i, j} = 1 && \forall i \in V \\
y_j - (n-1)x_{i, j} \geq y_i - (n-2) && \forall i \in V \setminus \{0\}, j\in V\setminus \{0, i\} \\
x_{i, j} \in \{0, 1\} && \forall i\in V, j \in V \\
y_i \geq 0 && \forall i \in V
\end{align}

In [ ]:
# Número de nodos y vértices
n, V = len(points), set(range(len(points)))
print(f'Número de nodos:\tn = {n}\nVértices:\tV = {V}')

In [ ]:
model = Model()

# Variables binarias que indican si se toma el camino de la ciudad i a la j
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# Variables continuas para evitar subrutas
y = [model.add_var() for i in V]

$$\sum_{i\in V, j\in V} d_{i, j}x_{i, j}$$


In [ ]:
# Función objetivo: 
model.objective = minimize(xsum(d[i][j]*x[i][j] for i in V for j in V))

$$\sum_{j\in V\setminus \{i\}} x_{i, j} = 1 \text{, } \forall i \in V$$

In [ ]:
# Restricción : Sal de cada ciudad una única vez
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

$$\sum_{i\in V\setminus \{j\}} x_{i, j} = 1 \text{, } \forall j \in V$$

In [ ]:
# Restricción : Entra a cada ciudad una única vez
for j in V:
    model += xsum(x[i][j] for i in V - {j}) == 1

$$y_j - (n-1)x_{i, j} \geq y_i - (n-2) \text{, } \forall i \in V \setminus \{0\}, j\in V\setminus \{0, i\}$$

In [ ]:
# Elimina subrutas
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[j] - (n-1)*x[i][j] >= y[i]-(n-2)

In [ ]:
# Optimizar
model.optimize()

# Revisar si se encontró una solución
edges = []
if model.num_solutions:
    print('Ruta con distancia total %g encontrada: %s'
              % (model.objective_value, points.index[0]))
    nc = 0 # Indice de la ciudad actual
    while True:
        oc = nc 
        nc = [i for i in V if x[nc][i].x >= 0.99][0] # Indice de la ciudad a la que se siguió
        edges.append((oc, nc)) # Guardamos el camino a tomar
        print(' -> %s' % points.index[nc]) # Imprimimos el siguiente paso de la ruta
        if nc == 0:  # Si regresa al inicio terminamos el ciclo
            break
    print('\n')

In [ ]:
plot_cities_and_route(points, edges)

## Datos Reales

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Point 

In [ ]:
#import data from file
url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5780421/bin/41598_2018_19772_MOESM3_ESM.xlsx'
data = pd.read_excel(url)
print(data.shape)
data.head()

In [ ]:
geometry = [Point(xy) for xy in zip(data['Longitude'],data['Latitude'])]
crs = {'init' : 'epsg:4326'}
geo_df = gpd.GeoDataFrame(data, crs = crs, geometry = geometry)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
conts = world.loc[world.continent.isin(['Asia','Europe'])]
boundaries = conts['geometry']

fig , ax = plt.subplots(figsize=(15,15))
boundaries.plot(ax=ax, alpha=0.8)
geo_df[geo_df['Longitude'] > 90 ].plot(ax=ax, marker='o', color='red', markersize=50);
geo_df[geo_df['Longitude'] <= 90 ].plot(ax=ax, marker='o', color='green', markersize=50);
plt.xlim(-25,150)

In [ ]:
def Haversine(A,B):
    """
    This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, 
    the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points 
    (ignoring any hills they fly over, of course!).
    Haversine
    formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    c = 2 ⋅ atan2( √a, √(1−a) )
    d = R ⋅ c
    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    note that angles need to be in radians to pass to trig functions!
    """
    lat1,lon1,lat2,lon2 = A[0],A[1],B[0],B[1]
    
    R = 6378.0088
    lat1,lon1,lat2,lon2 = map(np.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) **2
    c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

In [ ]:
location = data[['Latitude','Longitude']].drop_duplicates(subset=['Latitude','Longitude'])
location = location[location['Longitude'] <= 90].copy()
location.columns = ['x', 'y']
n = location.shape[0] #number of locations
print(n)
location.index = [f'ciudad_{i}' for i in range(n)]
print(location)

In [ ]:
mid_points = get_middle_points(location)
mid_points

In [ ]:
d = np.zeros((n,n))  # Distancia entre las ciudades
for i in range(n):
  for j in range(n):
    d[i,j] = Haversine(location.iloc[i], location.iloc[j])

d[:5, :5]

## Visualización de los datos

In [ ]:
plot_cities_and_route(location, product(range(n), range(n))) # Visualización de las ciudades

## TSP (Travelling salesman problem) con programación lineal

In [ ]:
# Número de nodos y vértices
n, V = len(location), set(range(len(location)))
print(f'Número de nodos:\tn = {n}\nVértices:\tV = {V}')

In [ ]:
model = Model()

# Variables binarias que indican si se toma el camino de la ciudad i a la j
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# Variables continuas para evitar subrutas
y = [model.add_var() for i in V]

In [ ]:
# Función objetivo: 
model.objective = minimize(xsum(d[i][j]*x[i][j] for i in V for j in V))

In [ ]:
# Restricción : Sal de cada ciudad una única vez
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

In [ ]:
# Restricción : Entra a cada ciudad una única vez
for j in V:
    model += xsum(x[i][j] for i in V - {j}) == 1

In [ ]:
# Elimina subrutas
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[j] - (n-1)*x[i][j] >= y[i]-(n-2)

In [ ]:
# Optimizar
model.optimize()

# Revisar si se encontró una solución
edges = []
if model.num_solutions:
    print('Ruta con distancia total %g encontrada: %s'
              % (model.objective_value, location.index[0]))
    nc = 0 # Indice de la ciudad actual
    while True:
        oc = nc 
        nc = [i for i in V if x[nc][i].x >= 0.99][0] # Indice de la ciudad a la que se siguió
        edges.append((oc, nc)) # Guardamos el camino a tomar
        print(' -> %s' % location.index[nc]) # Imprimimos el siguiente paso de la ruta
        if nc == 0:  # Si regresa al inicio terminamos el ciclo
            break
    print('\n')

In [ ]:
plot_cities_and_route(location, edges)

# Tarea

*   Hacer lo anterior en una sola función. El input es los puntos de localización y el output la ruta con distancia mínima

*   Utilizar la función para obtener una ruta para los datos con longitud mayor a 90


